In [23]:
import os
import sys
sys.path.append('/home/opencv/src/models/research')
sys.path.append('/home/opencv/src/models/research/slim/')
if 'PYTHONPATH' not in os.environ:
    os.environ['PYTHONPATH'] = ""
os.environ['PYTHONPATH'] += ':/home/opencv/src/models/research:/home/opencv/src/models/research/slim/'
os.environ['PYTHONPATH'] += ':/home/opencv/src/models/research:/home/opencv/src/models/research/'


In [4]:
# For the fruit model included in the repo below we have 240 training images
# For faster training time, images should be resized to 300x300 and then annotated
# Images should contain the objects of interest at various scales, angles, lighting conditions, locations
# For acceptable results - mAP@0.5 of 0.9 the model was trained with batch size of 24
# and 5000 steps. this takes about 1h using 2 augmentations. 
# using 5 augmentations it takes about 2h 
#num_steps = 5000  # A step means using a single batch of data. larger batch, less steps required
num_steps = 5000  # A step means using a single batch of data. larger batch, less steps required
#Number of evaluation steps.
num_eval_steps = 50 #50
#Batch size 24 is a setting that generally works well. can be changed higher or lower 
MODELS_CONFIG = {
        'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 24
    }
}
selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colab's GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [5]:
%cd /home/opencv/src/models/research


/home/opencv/src/models/research


In [6]:
!protoc object_detection/protos/*.proto --python_out=.

!python object_detection/builders/model_builder_test.py

object_detection/protos/input_reader.proto:5:1: warning: Import object_detection/protos/image_resizer.proto is unused.


In [7]:
#%%capture
repo_dir_path = '/home/opencv/src/object_detection_demo_flow'
%cd {repo_dir_path}
print(os.getcwd())

# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

# Set the paths
test_record_fname = '/home/opencv/src/object_detection_demo_flow/data/annotations/test.record'
train_record_fname = '/home/opencv/src/object_detection_demo_flow/data/annotations/train.record'
label_map_pbtxt_fname = '/home/opencv/src/object_detection_demo_flow/data/annotations/label_map.pbtxt'

/home/opencv/src/object_detection_demo_flow
/home/opencv/src/object_detection_demo_flow
Successfully converted xml to csv.
Generate `data/annotations/label_map.pbtxt`
Successfully converted xml to csv.


W0831 16:26:08.900258 139826120128320 module_wrapper.py:139] From generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0831 16:26:08.911897 139826120128320 module_wrapper.py:139] From generate_tfrecord.py:53: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /home/opencv/src/object_detection_demo_flow/data/annotations/train.record


W0831 16:26:10.567761 139873034827584 module_wrapper.py:139] From generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0831 16:26:10.575489 139873034827584 module_wrapper.py:139] From generate_tfrecord.py:53: The name tf.gfile.GFile is deprecated. Please use 

In [8]:
import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/home/opencv/src/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/home/opencv/src/models/research/pretrained_model
total 135M
drwxr-xr-x  3 opencv opencv 4.0K  3月 30  2018 .
drwxrwxr-x 64 opencv opencv 4.0K  8月 31 16:26 ..
-rw-r--r--  1 opencv opencv   77  3月 30  2018 checkpoint
-rw-r--r--  1 opencv opencv  67M  3月 30  2018 frozen_inference_graph.pb
-rw-r--r--  1 opencv opencv  65M  3月 30  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 opencv opencv  15K  3月 30  2018 model.ckpt.index
-rw-r--r--  1 opencv opencv 3.4M  3月 30  2018 model.ckpt.meta
-rw-r--r--  1 opencv opencv 4.2K  3月 30  2018 pipeline.config
drwxr-xr-x  3 opencv opencv 4.0K  3月 30  2018 saved_model


In [9]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/home/opencv/src/models/research/pretrained_model/model.ckpt'

# Configuring a Training Pipeline

In [10]:
import os
pipeline_fname = os.path.join('/home/opencv/src/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [11]:
import re
iou_threshold = 0.50
num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    # Set number of classes num_classes.
    s = re.sub('iou_threshold: [0-9].[0-9]+',
               'iou_threshold: {}'.format(iou_threshold), s)
    
    f.write(s)

# Train the model

In [12]:
from datetime import datetime
timestamp = datetime.timestamp(datetime.now())
dt_object = datetime.fromtimestamp(timestamp)
training_folder = "training_"+str(dt_object.year)+"_"+str(dt_object.month)+"_"+str(dt_object.day)
print("Training_folder: " + training_folder)


Training_folder: training_2020_8_31


In [11]:
# necessary
#*********************** change folder name adding timestamp TODO *************
model_dir = training_folder

# remove previous training result
!rm -rf ~/.nv
# !rm -rf {model_dir}
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# Optionally remove content in output model directory for a fresh start.
# !rm -rf {model_dir}
# os.makedirs(model_dir, exist_ok=True)
!python /home/opencv/src/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

W0831 15:49:00.170296 139811989407552 model_lib.py:717] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 5000
I0831 15:49:00.170428 139811989407552 config_util.py:552] Maybe overwriting train_steps: 5000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0831 15:49:00.170494 139811989407552 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0831 15:49:00.170548 139811989407552 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0831 15:49:00.170602 139811989407552 config_util.py:552] Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
I0831 15:49:00.170654 139811989407552 config_util.py:552] Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
I0831 15:49:00.170704 139811989407552 config_util.py:5

Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
W0831 15:49:15.436146 139811989407552 api.py:332] From /home/opencv/anaconda2/envs/ssd/lib/python3.6/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:1004: sample_distorted_bounding_box (from tensorflow.python.ops.image_ops_impl) is deprecated and will be removed in a future version.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use `tf.cast` instead.
W0831 15:49:18.414084 139811989407552 deprecation.py:323] From /home/opencv/src/models/research/object_detection/inputs.py:258: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
I0831 15:49:21.165728 139811989407552 estimator.py:1148] Calling model_fn.
Instructions for updating:
Please use `layer.__call__`

INFO:tensorflow:Running local_init_op.
I0831 15:49:41.046592 139811989407552 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0831 15:49:41.477147 139811989407552 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into training_2020_8_31/model.ckpt.
I0831 15:49:50.263451 139811989407552 basic_session_run_hooks.py:606] Saving checkpoints for 0 into training_2020_8_31/model.ckpt.
2020-08-31 15:49:59.254423: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-08-31 15:50:00.572949: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
INFO:tensorflow:loss = 5.561652, step = 0
I0831 15:50:02.427942 139811989407552 basic_session_run_hooks.py:262] loss = 5.561652, step = 0
INFO:tensorflow:global_step/sec: 3.53491
I0831 15:50:30.716584 139811989407552 basic_session_run_hooks.py:69

INFO:tensorflow:global_step/sec: 4.2598
I0831 15:59:51.889398 139811989407552 basic_session_run_hooks.py:692] global_step/sec: 4.2598
INFO:tensorflow:loss = 0.5079031, step = 2500 (23.475 sec)
I0831 15:59:51.889956 139811989407552 basic_session_run_hooks.py:260] loss = 0.5079031, step = 2500 (23.475 sec)
INFO:tensorflow:Saving checkpoints for 2503 into training_2020_8_31/model.ckpt.
I0831 15:59:52.353366 139811989407552 basic_session_run_hooks.py:606] Saving checkpoints for 2503 into training_2020_8_31/model.ckpt.
INFO:tensorflow:Calling model_fn.
I0831 15:59:54.354270 139811989407552 estimator.py:1148] Calling model_fn.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0831 15:59:56.039982 139811989407552 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0831 15:59:56.062242 139811989407552 convolutional_box_predictor.py:156] depth of additional conv before box predicto

INFO:tensorflow:Running local_init_op.
I0831 15:59:58.307283 139811989407552 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0831 15:59:58.427875 139811989407552 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Performing evaluation on 3 images.
I0831 16:00:00.687138 139806568118016 coco_evaluation.py:237] Performing evaluation on 3 images.
creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
I0831 16:00:00.687955 139806568118016 coco_tools.py:116] Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.00s)
I0831 16:00:00.688322 139806568118016 coco_tools.py:138] DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.122
 Average Precision  (AP) @[ IoU=0.50      | area=   a

INFO:tensorflow:global_step/sec: 4.27989
I0831 16:05:51.450875 139811989407552 basic_session_run_hooks.py:692] global_step/sec: 4.27989
INFO:tensorflow:loss = 0.40116644, step = 4000 (23.365 sec)
I0831 16:05:51.451458 139811989407552 basic_session_run_hooks.py:260] loss = 0.40116644, step = 4000 (23.365 sec)
INFO:tensorflow:global_step/sec: 4.2847
I0831 16:06:14.789731 139811989407552 basic_session_run_hooks.py:692] global_step/sec: 4.2847
INFO:tensorflow:loss = 0.43112257, step = 4100 (23.339 sec)
I0831 16:06:14.790338 139811989407552 basic_session_run_hooks.py:260] loss = 0.43112257, step = 4100 (23.339 sec)
INFO:tensorflow:global_step/sec: 4.28698
I0831 16:06:38.116192 139811989407552 basic_session_run_hooks.py:692] global_step/sec: 4.28698
INFO:tensorflow:loss = 0.40563902, step = 4200 (23.326 sec)
I0831 16:06:38.116811 139811989407552 basic_session_run_hooks.py:260] loss = 0.40563902, step = 4200 (23.326 sec)
INFO:tensorflow:global_step/sec: 4.27836
I0831 16:07:01.489608 139811989

INFO:tensorflow:Running local_init_op.
I0831 16:09:50.834153 139811989407552 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0831 16:09:50.950393 139811989407552 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Performing evaluation on 3 images.
I0831 16:09:52.738413 139807997032192 coco_evaluation.py:237] Performing evaluation on 3 images.
creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
I0831 16:09:52.738900 139807997032192 coco_tools.py:116] Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.00s)
I0831 16:09:52.739249 139807997032192 coco_tools.py:138] DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.099
 Average Precision  (AP) @[ IoU=0.50      | area=   a

INFO:tensorflow:Assets added to graph.
I0831 16:09:55.555865 139811989407552 builder_impl.py:665] Assets added to graph.
INFO:tensorflow:No assets to write.
I0831 16:09:55.556006 139811989407552 builder_impl.py:460] No assets to write.
INFO:tensorflow:SavedModel written to: training_2020_8_31/export/Servo/temp-b'1598857792'/saved_model.pb
I0831 16:09:56.121483 139811989407552 builder_impl.py:425] SavedModel written to: training_2020_8_31/export/Servo/temp-b'1598857792'/saved_model.pb
INFO:tensorflow:Loss for final step: 0.40447065.
I0831 16:09:56.486075 139811989407552 estimator.py:371] Loss for final step: 0.40447065.


In [13]:
!pwd

/home/opencv/src/object_detection_demo_flow


Generate graph

In [15]:
import re
import numpy as np

output_directory = "frozen_"+str(dt_object.year)+"_"+str(dt_object.month)+"_"+str(dt_object.day)
print("Frozen_folder: " + output_directory)
#output_directory = './fine_tuned_model'
# output_directory = '/content/gdrive/My\ Drive/data/'


Frozen_folder: frozen_2020_8_31


In [18]:
print("start saving")
model_dir = training_folder
lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

start saving


In [24]:
last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /src/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

print("saving end")

training_2020_8_31/model.ckpt-5000
python: can't open file '/src/models/research/object_detection/export_inference_graph.py': [Errno 2] No such file or directory
saving end


In [22]:
print(os.environ['PYTHONPATH'])

/opt/intel/openvino_2020.1.023/python/python3.6:/opt/intel/openvino_2020.1.023/python/python3:/opt/intel/openvino_2020.1.023/deployment_tools/open_model_zoo/tools/accuracy_checker:/opt/intel/openvino_2020.1.023/deployment_tools/model_optimizer::/home/opencv/src/models/research:/home/opencv/src/models/research/slim/:/home/opencv/src/models/research:/home/opencv/src/models/research/slim/
